In [1]:
from herbie import Herbie
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [126]:
#Get dates to get weather model data for
unique_dates = pd.to_datetime(pd.read_csv('cleaned_mwobs.csv')['valid']).dt.date.unique()

In [132]:
#Format dates as strings
formatted_dates = []
for i in range(len(unique_dates)):
  formatted_dates.append(unique_dates[i].strftime("%Y-%m-%d") + " 10:00")

In [133]:
#Get coordinates closest to Mt Washington
#distances = (H.xarray(':TMP:2 m').t2m[770:900, 1450:1630].longitude - 288.6967)**2 + (H.xarray(':TMP:2 m').t2m[770:900, 1450:1630].latitude - 44.2706)**2
#np.argwhere(~np.isnan(distances.where(distances==distances.min()).values))


In [136]:
#Terrible function that I hard coded but it works
#Gets forecast data from average of 6 100_km regions around Mt Washington as well as Mt Washington itself
def make_data_point(tempdata, dptdata, uwinddata, vwinddata, wspddata, precipdata):
  return np.array([tempdata[842, 1589].values,
  tempdata[842:842+30, 1589-33:1589].values.mean(),
  tempdata[842:842+30, 1589-66:1589-33].values.mean(),
  tempdata[842-30:842, 1589-33:1589].values.mean(),
  tempdata[842-30:842, 1589-66:1589-33].values.mean(),
  tempdata[842-30:842, 1589:1589+33].values.mean(),
  tempdata[842:842+30, 1589:1589+33].values.mean(),
  dptdata[842, 1589].values,
  dptdata[842:842+30, 1589-33:1589].values.mean(),
  dptdata[842:842+30, 1589-66:1589-33].values.mean(),
  dptdata[842-30:842, 1589-33:1589].values.mean(),
  dptdata[842-30:842, 1589-66:1589-33].values.mean(),
  dptdata[842-30:842, 1589:1589+33].values.mean(),
  dptdata[842:842+30, 1589:1589+33].values.mean(),
  uwinddata[842, 1589].values,
  uwinddata[842:842+30, 1589-33:1589].values.mean(),
  uwinddata[842:842+30, 1589-66:1589-33].values.mean(),
  uwinddata[842-30:842, 1589-33:1589].values.mean(),
  uwinddata[842-30:842, 1589-66:1589-33].values.mean(),
  uwinddata[842-30:842, 1589:1589+33].values.mean(),
  uwinddata[842:842+30, 1589:1589+33].values.mean(),
  vwinddata[842, 1589].values,
  vwinddata[842:842+30, 1589-33:1589].values.mean(),
  vwinddata[842:842+30, 1589-66:1589-33].values.mean(),
  vwinddata[842-30:842, 1589-33:1589].values.mean(),
  vwinddata[842-30:842, 1589-66:1589-33].values.mean(),
  vwinddata[842-30:842, 1589:1589+33].values.mean(),
  vwinddata[842:842+30, 1589:1589+33].values.mean(),
  wspddata[842, 1589].values,
  wspddata[842:842+30, 1589-33:1589].values.mean(),
  wspddata[842:842+30, 1589-66:1589-33].values.mean(),
  wspddata[842-30:842, 1589-33:1589].values.mean(),
  wspddata[842-30:842, 1589-66:1589-33].values.mean(),
  wspddata[842-30:842, 1589:1589+33].values.mean(),
  wspddata[842:842+30, 1589:1589+33].values.mean(),
  precipdata[842, 1589].values,
  precipdata[842:842+30, 1589-33:1589].values.mean(),
  precipdata[842:842+30, 1589-66:1589-33].values.mean(),
  precipdata[842-30:842, 1589-33:1589].values.mean(),
  precipdata[842-30:842, 1589-66:1589-33].values.mean(),
  precipdata[842-30:842, 1589:1589+33].values.mean(),
  precipdata[842:842+30, 1589:1589+33].values.mean(),
  ])

In [ ]:
all_data_points = []
for date in formatted_dates:
  print(date)
  H = Herbie(
      formatted_dates[i],
      model="hrrr",
      product="sfc",
      fxx=10,
  )

  print('Collecting data...')
  #Get relevant fields from weather model
  tempdata = H.xarray(':TMP:2 m').t2m
  dptdata = H.xarray(':DPT:2 m').d2m
  uwinddata = H.xarray(':UGRD:10 m').u10
  vwinddata = H.xarray(':VGRD:10 m').v10
  wspddata = np.sqrt(uwinddata**2 + vwinddata**2)
  precipdata = H.xarray(':APCP:').tp

  #Get data point from fields
  data_point = make_data_point(tempdata, dptdata, uwinddata, vwinddata, wspddata, precipdata)
  all_data_points.append(data_point)


In [144]:
#Save results once finished
np.savetxt('model_data.csv', all_data_points)